In [1]:
# import the required libraries
import docx2txt
import os
import re
import csv
import pandas as pd
import numpy as np

In [2]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )

folderlink = '..//data//'
input_folder = 'input//'
input_file = os.path.join(folderlink+input_folder, 'Repertoriummetoudepersoonsnummers.docx')
folder_output = 'output//'
output_txt = folderlink+folder_output+'output.txt'
output_csv = folderlink+folder_output+'output_file.csv'

In [3]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
# Converting the docx to a text file and remove all unecessary rows.

# Use docx2txt library to extract text from .docx file
text = docx2txt.process(input_file)

# Remove excessive whitespaces
data = ' '.join(text.split())

Manual updates in the dataset:

1.  FROM: "N.N. "de oude vicarius">pred. Lichtenvoorde 1602 tot 1615.<20871>"
    TO: "N.N. "de oude vicarius" pred. Lichtenvoorde 1602 tot 1615.<20871>" 
    REASON: since it contained the > character which is used to define ID fields. 
 
2.  FROM: "Bosch; Cornelius Geb. Utrecht 1634; pred. Renswoude 16 dec. 1656, Maasland 15 april 1663, Brielle 30 jan. 1667, Alkmaar 1667, 's Gravenhage 5 juli 1676, emer. 1713,;overl. 28 maart 1715.<1185>"
    TO:Bosch; Cornelius
Geb. Utrecht 1634; pred. Renswoude 16 dec. 1656, Maasland 15 april 1663, Brielle 30 jan. 1667, Alkmaar 1667, 's Gravenhage 5 juli 1676, emer. 1713, overl. 28 maart 1715.<1185>
    REASON: Since it splitted the string based on ",;overl"

3.  FROM: Leeuwen, van, Cornelis [z.v. Cornelis]
Geb. Hazerswoude 1611; pred. Boskoop en Middelburg 1637, overl. 1681.<5778>
    TO: Leeuwen, van; Cornelis [z.v. Cornelis]
Geb. Hazerswoude 1611; pred. Boskoop en Middelburg 1637, overl. 1681.<5778>
    REASON: there was no ; between the name and surname (including infix)

4.  FROM: Peenen, van, Marcus
Gedoopt Leiden 31 aug. 1642; pred. Koudekerk aan den Rijn 2 sept. 1668, Leiden 1680, begraven 1 febr. 1696.<7388>
    TO: Peenen, van; Marcus
Gedoopt Leiden 31 aug. 1642; pred. Koudekerk aan den Rijn 2 sept. 1668, Leiden 1680, begraven 1 febr. 1696.<7388>
    REASON: there was no ; between the name and surname (including infix)

5.  FROM: Knuyt, de (Kuntius), Elias
Geb. Middelburg yyy; pred. Oude Niedorp en Veenhuizen (NH) mei 1628, Sint Annaland 1630, Westkapelle 7 maart 1641, overl. --of vertrokken?-- 1642.<5381>
    TO: Knuyt, de (Kuntius); Elias
Geb. Middelburg yyy; pred. Oude Niedorp en Veenhuizen (NH) mei 1628, Sint Annaland 1630, Westkapelle 7 maart 1641, overl. --of vertrokken?-- 1642.<5381>
    REASON: there was no ; between the name and surname (including infix)

6.  FROM: Leonardis, de, Paulus
Geb. Keulen yyy; pred. Bacharach (Pfalz) 16.., Kampen 1620, overl. 1649.<5836>
    TO: Leonardis, de; Paulus
Geb. Keulen yyy; pred. Bacharach (Pfalz) 16.., Kampen 1620, overl. 1649.<5836>
    REASON: there was no ; between the name and surname (including infix)

7.  FROM: Tamerus; Henricus
Geb. xxx ca. 1540; voorheen Lutheraan;--onwettig pred. Heteren en Randwijk ca. 1600-1602; pred. Eethen en Meeuwen 1606, Doeveren, Gansoyen en Genderen 1610, afgezet als remonstrant 1619; overl. Heusden.<21206>
    TO: Tamerus; Henricus
Geb. xxx ca. 1540; voorheen Lutheraan; onwettig pred. Heteren en Randwijk ca. 1600-1602; pred. Eethen en Meeuwen 1606, Doeveren, Gansoyen en Genderen 1610, afgezet als remonstrant 1619; overl. Heusden.<21206>
    REASON: -- conflicted created a line break. 




In [5]:
data = data.replace('overl. 28 maart 1715.<1185>', ' overl. 28 maart 1715.<1185>').replace('>pred.', ' pred.').replace('Leeuwen, van, Cornelis [z.v. Cornelis]','Leeuwen, van; Cornelis [z.v. Cornelis]',).replace('Peenen, van, Marcus','Peenen, van; Marcus').replace('Knuyt, de (Kuntius), Elias','Knuyt, de (Kuntius); Elias').replace('Leonardis, de, Paulus','Leonardis, de; Paulus').replace('Lutheraan;--onwettig','Lutheraan; onwettig')

In [6]:
# Replace semicolons with newlines and add semicolons around < and > since these identify the IDs
data = data.replace(';', ';\n').replace(';\n ', '; ').replace('>', '>;\n ').replace('<', ';<')
lines = data.split('\n')

lines = [line for line in lines if not line.startswith('; ;') and not line.startswith('; ')]
data = '\n'.join(lines)
lines = data.strip().split('\n')
data = '\n'.join([line.lstrip() for line in lines])


In [7]:
def replace_pred_count(string):
    count = 0
    result = ""
    position = string.find(" pred.")

    while position != -1:
        result += string[:position] + str(count) + " pred."
        string = string[position + len(" pred."):]
        count += 1
        position = string.find(" pred.")

    result += string

    return result

In [8]:
lines = data.split('\n')

with open(output_txt, "w", encoding='utf-8') as file:
    for line in lines:
        result_string = replace_pred_count(line)
        file.write(result_string + "\n")

In [9]:
columns = ("Geb.", # deze is cruciaal
 "0 pred.",
 "1 pred.",
 "2 pred.",
 "3 pred.",
 "4 pred.",
 "overl.", # deze is cruciaal
 "Gedoopt", # deze is cruciaal
 "legerpred.",
 "pastoor",
 "garnizoenspred.",
 "emer.",
 "begraven",
 "conrector",
 " rector",
 "monnik",           
 "schoolmeester",
 "hoogleraar",
 "chirurgijn",
 "praeceptor",
 "ziekentrooster",
 "vlootpred.",
 "legerpred.",
 "ambassadepred."           
)          
    

In [10]:
for column in columns:
    with open(output_txt, 'r', encoding='utf-8') as f:
        lines = f.readlines()        
    with open(output_txt,'w', encoding='utf-8') as f:
        for line in lines:
            if "; "+column in line:
                f.write(line)
            elif column in line:
                line = line.replace(column, ";"+column)
                f.write(line)
            else:
                f.write(line)

In [11]:
# Define the headers for the output file
headers = ['surname_temp', 'name', 'Field1', 'Field2', 'Field3', 'Field4', 'Field5', 'Field6', 'Field7', 'Field8', 'Field9', 'Field10', 'Field11','Field12','Field13','Field14','Field15','Field16','Field17']

with open(output_txt, 'r', encoding='utf-8') as infile, open(output_csv, 'w', newline='', encoding='utf-8' ) as outfile:
    reader = csv.reader(infile, delimiter=';')
    writer = csv.writer(outfile, delimiter=';')
    
    # Write the headers to the output file
    writer.writerow(headers)

    # Loop through each row in the input file and write it to the output file with 12 fields
    for row in reader:
        # Create a new row with 12 fields by extending the current row with empty values
        new_row = row + [''] * (12 - len(row))
        writer.writerow(new_row)

In [12]:
df = pd.read_csv(output_csv, sep=';', encoding='utf-8')

In [13]:
#In the file all IDs are stored between < and > therefore 
for column in df.columns:
    # Check if any value in the column contains '<'
    if df[column].astype(str).str.contains('<').any():
        # Copy the values containing '<' to the 'test' column
        df.loc[df[column].astype(str).str.contains('<'), 'id'] = df[column]


In [14]:
df['id'] = df['id'].str.replace('>', '').str.replace('<','')

In [15]:
for column in columns:
    df[column] = df.apply(lambda row: row[row.astype(str).str.contains(column)].iloc[0] if any(row.astype(str).str.contains(column)) else '', axis=1)

In [16]:
df['original_input'] = df['surname_temp'].fillna('') + df['name'].fillna('') + df['Field1'].fillna('') + df['Field2'].fillna('')+ df['Field3'].fillna('')+ df['Field4'].fillna('')+ df['Field5'].fillna('')+ df['Field6'].fillna('')+ df['Field7'].fillna('')+ df['Field8'].fillna('')+ df['Field9'].fillna('')+ df['Field10'].fillna('')+ df['Field11'].fillna('')+ df['Field12'].fillna('')+ df['Field13'].fillna('')+ df['Field14'].fillna('')+ df['Field15'].fillna('')+ df['Field16'].fillna('')+ df['Field17'].fillna('')
df['original_input'] = df['original_input'].str.replace('0 pred. ', ' pred. ').str.replace('1  pred.',' pred. ').str.replace('2 pred.',' pred. ').str.replace('3  pred.',' pred. ').str.replace('4  pred.',' pred. ')

In [17]:
for column in columns:
    df[column] = df[column].str.replace(column, '')

In [18]:
df['name_info_family'] =df['name'] .str.extract(r'\[(.*?)\]')


In [19]:
df['nickname'] =df['surname_temp'] .str.extract(r'\((.*?)\)')


In [20]:
columns_rename = {
    'Geb.': 'birth', 
    #' pred.': 'minister', 
    'overl.': 'death', 
    'Gedoopt':'baptized', 
    'legerpred.':'legerpredikant',
    'pastoor':'pastoor',
    'garnizoenspred.':'garnizoenspredikant',
    "emer.":'emeritus_status',
    "begraven":'burried',
    "conrector":'conrector',
    " rector":'rector',
    "monnik":'monnik',           
    "schoolmeester":'schoolmeester',
    "hoogleraar":'hoogleraar',
    "chirurgijn":'chirurgijn',
    "praeceptor":'praeceptor',
    "ziekentrooster":'ziekentrooster',
    "vlootpred.":'vlootpredikant',
    "legerpred.":'legerpredikant',
    "ambassadepred.":'ambassadepredikant'}

In [21]:
# Rename the columns
df = df.rename(columns=columns_rename)
new_columns = list(columns_rename.values())


In [22]:
df.head()

,surname_temp,name,Field1,Field2,Field3,Field4,Field5,Field6,Field7,Field8,Field9,Field10,Field11,Field12,Field13,Field14,Field15,Field16,Field17,id,birth,0 pred.,1 pred.,2 pred.,3 pred.,4 pred.,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,nickname
0,"Aalburg, van",Johannes,Geb. Zierikzee ca. 1717,NaN,"0 pred. Oudkarspel 30 juni 1743,",overl. 14 maart 1777.,<1>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Zierikzee ca. 1717,"Oudkarspel 30 juni 1743,",,,,,14 maart 1777.,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN
1,Aalst,Wilhelmus,Gedoopt Biggekerke 5 jan. 1664,NaN,"0 pred. Aardenburg 22 mei 1695,",overl. 19 dec. 1700.,<4>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,,"Aardenburg 22 mei 1695,",,,,,19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN
2,"Aalst, van",Cornelius,Geb. Castricum ca. 1686,ambassadepred. in Parijs maart tot dec. 1715,NaN,"0 pred. Kalslagen ber. 21 febr. 1717,",emer. 1751,overl. Amsterdam 27 aug. 1756.,<2>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Castricum ca. 1686,"Kalslagen ber. 21 febr. 1717,",,,,,Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN
3,"Aalst, van",Gerardus,Geb. xxx sept. 1678,NaN,"0 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715,",emer. 1755,overl. 29 juni 1759.,<3>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,xxx sept. 1678,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715,",,,,,29 juni 1759.,,,,,1755,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN
4,Aalstius,Henricus,Geb. 's-Hertogenbosch (?) yyy,NaN,"0 pred. Castricum en Heemskerk nov. 1700,",emer. sept. 1733,overl. Limmen 15 maart 1736.,<5>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,'s-Hertogenbosch (?) yyy,"Castricum en Heemskerk nov. 1700,",,,,,Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN


In [23]:
array_drop = [i for i in range(1, 18)]
for dropid in array_drop:
    column_dropid = 'Field'+str(dropid)
    df = df.drop(column_dropid, axis=1)

In [24]:
def extract_year(text):
    match = re.search(r'\d{4}', text)
    if match:
        return match.group(0)
    else:
        return None

In [25]:
df['minister'] = df['0 pred.']+ ','+df['1 pred.']+ ','+df['2 pred.']+ ','+df['3 pred.']+ ','+df['4 pred.']

In [26]:
df = df.drop(['0 pred.','1 pred.','2 pred.','3 pred.','4 pred.'], axis=1)

In [27]:
function_year = [word for word in new_columns if word != 'minister']

print(function_year)

['birth', 'death', 'baptized', 'legerpredikant', 'pastoor', 'garnizoenspredikant', 'emeritus_status', 'burried', 'conrector', 'rector', 'monnik', 'schoolmeester', 'hoogleraar', 'chirurgijn', 'praeceptor', 'ziekentrooster', 'vlootpredikant', 'ambassadepredikant']


In [28]:
df.head()

,surname_temp,name,id,birth,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,nickname,minister
0,"Aalburg, van",Johannes,1,Zierikzee ca. 1717,14 maart 1777.,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN,"Oudkarspel 30 juni 1743, ,,,,"
1,Aalst,Wilhelmus,4,,19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN,"Aardenburg 22 mei 1695, ,,,,"
2,"Aalst, van",Cornelius,2,Castricum ca. 1686,Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN,"Kalslagen ber. 21 febr. 1717, ,,,,"
3,"Aalst, van",Gerardus,3,xxx sept. 1678,29 juni 1759.,,,,,1755,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, ,,,,"
4,Aalstius,Henricus,5,'s-Hertogenbosch (?) yyy,Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN,"Castricum en Heemskerk nov. 1700, ,,,,"


In [29]:
for year in function_year:
    fld_year = 'year_' +year
    df[fld_year] = df[year].apply(lambda x: extract_year(x))    

In [30]:
for year_accu in function_year:
    accu_fld_year = 'accu_year_' +year_accu
    df[accu_fld_year] = ''
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Check if the string contains "ca." (case-insensitive)
        if 'ca.' in row[year_accu].lower():
            # If found, set the value of the "accuracy" column to "circa"
            df.at[index, accu_fld_year] = 'circa'
            
    

In [31]:
months =(" januari ", 
 " februari ",
 " maart ",
 " april ",
 " mei ",
 " juni ",
 " juli ",
 " augustus ",
 " september ",
 " oktober ",
 " november ",           
 " december ",
 "jan. ",
 "feb. ",
 "mrt. ",
 "apr. ",
 "jun. ",
 "jul. ",
 "aug. ",
 "sept. ",
 "sep. ",
 "okt. ",
 "nov. ",
 "dec. ",
 "yyy",
 "xxx",
 "ca.",
 "febr."        
) 


In [32]:
for column_strip in function_year:   
    for month in months:
        df[column_strip] = df[column_strip].str.replace(month, '')


In [33]:
for column_strip in function_year:
    df[column_strip] = df[column_strip].apply(lambda x: re.sub(r'[\d\.]', '', x))

In [34]:
df.head()

,surname_temp,name,id,birth,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,nickname,minister,year_birth,year_death,year_baptized,year_legerpredikant,year_pastoor,year_garnizoenspredikant,year_emeritus_status,year_burried,year_conrector,year_rector,year_monnik,year_schoolmeester,year_hoogleraar,year_chirurgijn,year_praeceptor,year_ziekentrooster,year_vlootpredikant,year_ambassadepredikant,accu_year_birth,accu_year_death,accu_year_baptized,accu_year_legerpredikant,accu_year_pastoor,accu_year_garnizoenspredikant,accu_year_emeritus_status,accu_year_burried,accu_year_conrector,accu_year_rector,accu_year_monnik,accu_year_schoolmeester,accu_year_hoogleraar,accu_year_chirurgijn,accu_year_praeceptor,accu_year_ziekentrooster,accu_year_vlootpredikant,accu_year_ambassadepredikant
0,"Aalburg, van",Johannes,1,Zierikzee,,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN,"Oudkarspel 30 juni 1743, ,,,,",1717,1777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,circa,,,,,,,,,,,,,,,,,
1,Aalst,Wilhelmus,4,,,Biggekerke,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN,"Aardenburg 22 mei 1695, ,,,,",None,1700,1664,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,
2,"Aalst, van",Cornelius,2,Castricum,Amsterdam,,,,,,,,,,,,,,,,in Parijstot,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN,"Kalslagen ber. 21 febr. 1717, ,,,,",1686,1756,None,None,None,None,1751,None,None,None,None,None,None,None,None,None,None,1715,circa,,,,,,,,,,,,,,,,,
3,"Aalst, van",Gerardus,3,,,,,,,,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, ,,,,",1678,1759,None,None,None,None,1755,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,
4,Aalstius,Henricus,5,'s-Hertogenbosch (?),Limmen,,,,,,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN,"Castricum en Heemskerk nov. 1700, ,,,,",None,1736,None,None,None,None,1733,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,


In [35]:
df['surname_temp']= df['surname_temp'].str.replace(r'\(.*\)', '', regex=True)
df['name']= df['name'].str.replace(r'\[.*\]', '', regex=True)

In [36]:
df.head()

,surname_temp,name,id,birth,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,nickname,minister,year_birth,year_death,year_baptized,year_legerpredikant,year_pastoor,year_garnizoenspredikant,year_emeritus_status,year_burried,year_conrector,year_rector,year_monnik,year_schoolmeester,year_hoogleraar,year_chirurgijn,year_praeceptor,year_ziekentrooster,year_vlootpredikant,year_ambassadepredikant,accu_year_birth,accu_year_death,accu_year_baptized,accu_year_legerpredikant,accu_year_pastoor,accu_year_garnizoenspredikant,accu_year_emeritus_status,accu_year_burried,accu_year_conrector,accu_year_rector,accu_year_monnik,accu_year_schoolmeester,accu_year_hoogleraar,accu_year_chirurgijn,accu_year_praeceptor,accu_year_ziekentrooster,accu_year_vlootpredikant,accu_year_ambassadepredikant
0,"Aalburg, van",Johannes,1,Zierikzee,,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN,"Oudkarspel 30 juni 1743, ,,,,",1717,1777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,circa,,,,,,,,,,,,,,,,,
1,Aalst,Wilhelmus,4,,,Biggekerke,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN,"Aardenburg 22 mei 1695, ,,,,",None,1700,1664,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,
2,"Aalst, van",Cornelius,2,Castricum,Amsterdam,,,,,,,,,,,,,,,,in Parijstot,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN,"Kalslagen ber. 21 febr. 1717, ,,,,",1686,1756,None,None,None,None,1751,None,None,None,None,None,None,None,None,None,None,1715,circa,,,,,,,,,,,,,,,,,
3,"Aalst, van",Gerardus,3,,,,,,,,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, ,,,,",1678,1759,None,None,None,None,1755,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,
4,Aalstius,Henricus,5,'s-Hertogenbosch (?),Limmen,,,,,,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN,"Castricum en Heemskerk nov. 1700, ,,,,",None,1736,None,None,None,None,1733,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,


In [37]:
df[['surname', 'infix']] = df['surname_temp'].str.split(',', expand=True)

In [38]:
df.head()

,surname_temp,name,id,birth,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,nickname,minister,year_birth,year_death,year_baptized,year_legerpredikant,year_pastoor,year_garnizoenspredikant,year_emeritus_status,year_burried,year_conrector,year_rector,year_monnik,year_schoolmeester,year_hoogleraar,year_chirurgijn,year_praeceptor,year_ziekentrooster,year_vlootpredikant,year_ambassadepredikant,accu_year_birth,accu_year_death,accu_year_baptized,accu_year_legerpredikant,accu_year_pastoor,accu_year_garnizoenspredikant,accu_year_emeritus_status,accu_year_burried,accu_year_conrector,accu_year_rector,accu_year_monnik,accu_year_schoolmeester,accu_year_hoogleraar,accu_year_chirurgijn,accu_year_praeceptor,accu_year_ziekentrooster,accu_year_vlootpredikant,accu_year_ambassadepredikant,surname,infix
0,"Aalburg, van",Johannes,1,Zierikzee,,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN,"Oudkarspel 30 juni 1743, ,,,,",1717,1777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,circa,,,,,,,,,,,,,,,,,,Aalburg,van
1,Aalst,Wilhelmus,4,,,Biggekerke,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN,"Aardenburg 22 mei 1695, ,,,,",None,1700,1664,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,Aalst,None
2,"Aalst, van",Cornelius,2,Castricum,Amsterdam,,,,,,,,,,,,,,,,in Parijstot,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN,"Kalslagen ber. 21 febr. 1717, ,,,,",1686,1756,None,None,None,None,1751,None,None,None,None,None,None,None,None,None,None,1715,circa,,,,,,,,,,,,,,,,,,Aalst,van
3,"Aalst, van",Gerardus,3,,,,,,,,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, ,,,,",1678,1759,None,None,None,None,1755,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,Aalst,van
4,Aalstius,Henricus,5,'s-Hertogenbosch (?),Limmen,,,,,,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN,"Castricum en Heemskerk nov. 1700, ,,,,",None,1736,None,None,None,None,1733,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,Aalstius,None


In [39]:
df = df.drop(['surname_temp'], axis=1)

In [40]:
df['join_name'] = df['surname']+df['name']+df['infix'].fillna('')
df['join_name'] = df['join_name'].str.replace("  "," ")

In [41]:
df.head()

,name,id,birth,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,nickname,minister,year_birth,year_death,year_baptized,year_legerpredikant,year_pastoor,year_garnizoenspredikant,year_emeritus_status,year_burried,year_conrector,year_rector,year_monnik,year_schoolmeester,year_hoogleraar,year_chirurgijn,year_praeceptor,year_ziekentrooster,year_vlootpredikant,year_ambassadepredikant,accu_year_birth,accu_year_death,accu_year_baptized,accu_year_legerpredikant,accu_year_pastoor,accu_year_garnizoenspredikant,accu_year_emeritus_status,accu_year_burried,accu_year_conrector,accu_year_rector,accu_year_monnik,accu_year_schoolmeester,accu_year_hoogleraar,accu_year_chirurgijn,accu_year_praeceptor,accu_year_ziekentrooster,accu_year_vlootpredikant,accu_year_ambassadepredikant,surname,infix,join_name
0,Johannes,1,Zierikzee,,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN,"Oudkarspel 30 juni 1743, ,,,,",1717,1777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,circa,,,,,,,,,,,,,,,,,,Aalburg,van,Aalburg Johannes van
1,Wilhelmus,4,,,Biggekerke,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN,"Aardenburg 22 mei 1695, ,,,,",None,1700,1664,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,Aalst,None,Aalst Wilhelmus
2,Cornelius,2,Castricum,Amsterdam,,,,,,,,,,,,,,,,in Parijstot,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN,"Kalslagen ber. 21 febr. 1717, ,,,,",1686,1756,None,None,None,None,1751,None,None,None,None,None,None,None,None,None,None,1715,circa,,,,,,,,,,,,,,,,,,Aalst,van,Aalst Cornelius van
3,Gerardus,3,,,,,,,,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, ,,,,",1678,1759,None,None,None,None,1755,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,Aalst,van,Aalst Gerardus van
4,Henricus,5,'s-Hertogenbosch (?),Limmen,,,,,,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN,"Castricum en Heemskerk nov. 1700, ,,,,",None,1736,None,None,None,None,1733,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,Aalstius,None,Aalstius Henricus


In [42]:

birth_org = df[['id','birth','year_birth','accu_year_birth']]
birth_org.to_csv(folderlink+folder_output+'birth_info.csv', sep=';', encoding='utf-8', index=False)


In [43]:
death_org = df[['id','death','year_death','accu_year_death']]
death_org.to_csv(folderlink+folder_output+'death_info.csv', sep=';', encoding='utf-8', index=False)

In [44]:
bapt_org = df[['id','baptized','year_baptized','accu_year_baptized']]
bapt_org.to_csv(folderlink+folder_output+'bapt_info.csv', sep=';', encoding='utf-8', index=False)

In [45]:
df.to_csv(folderlink+folder_output+'parent_data.csv', sep=';', encoding='utf-8', index=False)

From here we are going to make the parent child relationship with predikanten Ministers

Here the minister data is set as a child. 

In [46]:
subset_pred = df[['id','minister','join_name','original_input']]

In [47]:
subset_pred.head()

,id,minister,join_name,original_input
0,1,"Oudkarspel 30 juni 1743, ,,,,",Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>"
1,4,"Aardenburg 22 mei 1695, ,,,,",Aalst Wilhelmus,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>"
2,2,"Kalslagen ber. 21 febr. 1717, ,,,,",Aalst Cornelius van,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>"
3,3,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, ,,,,",Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>"
4,5,"Castricum en Heemskerk nov. 1700, ,,,,",Aalstius Henricus,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>"


In [48]:
df_expanded = subset_pred.assign(minister=subset_pred['minister'].str.split(','))

# Explode the 'pred.' column to create separate rows for each item
df_expanded = df_expanded.explode('minister')

In [49]:
df_expanded.head()

,id,minister,join_name,original_input
0,1,Oudkarspel 30 juni 1743,Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>"
0,1,,Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>"
0,1,,Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>"
0,1,,Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>"
0,1,,Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>"


In [50]:
df_filtered = df_expanded[['id','minister','join_name','original_input']]

In [51]:
childs = df_filtered[(df_filtered["minister"] !=" ") & (df_filtered["minister"] !="")]

In [52]:
childs.head()

,id,minister,join_name,original_input
0,1,Oudkarspel 30 juni 1743,Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>"
1,4,Aardenburg 22 mei 1695,Aalst Wilhelmus,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>"
2,2,Kalslagen ber. 21 febr. 1717,Aalst Cornelius van,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>"
3,3,Vuren en Dalem 10 aug. 1704,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>"
3,3,Sommelsdijk 13 juni 1706,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>"


In [53]:
childs['minister_year'] = childs['minister'].apply(lambda x: extract_year(x))

C:\Windows\Temp\ipykernel_10528\3394215768.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs['minister_year'] = childs['minister'].apply(lambda x: extract_year(x))


In [54]:
accu_fld_year = 'accu_year_' +'minster'
childs[accu_fld_year] = ''
# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if the string contains "ca." (case-insensitive)
    if 'ca.' in row[year_accu].lower():
        # If found, set the value of the "accuracy" column to "circa"
        df.at[index, accu_fld_year] = 'circa'

C:\Windows\Temp\ipykernel_10528\322161388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs[accu_fld_year] = ''


In [55]:
childs.head()

,id,minister,join_name,original_input,minister_year,accu_year_minster
0,1,Oudkarspel 30 juni 1743,Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",1743,
1,4,Aardenburg 22 mei 1695,Aalst Wilhelmus,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",1695,
2,2,Kalslagen ber. 21 febr. 1717,Aalst Cornelius van,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",1717,
3,3,Vuren en Dalem 10 aug. 1704,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",1704,
3,3,Sommelsdijk 13 juni 1706,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",1706,


In [56]:
for month in months:
        childs['minister'] = childs['minister'].str.replace(month, '')

childs['minister'] = childs['minister'].apply(lambda x: re.sub(r'[\d\.]', '', x))

C:\Windows\Temp\ipykernel_10528\1973265549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs['minister'] = childs['minister'].str.replace(month, '')
C:\Windows\Temp\ipykernel_10528\1973265549.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs['minister'] = childs['minister'].apply(lambda x: re.sub(r'[\d\.]', '', x))


In [57]:
childs.head()

,id,minister,join_name,original_input,minister_year,accu_year_minster
0,1,Oudkarspel,Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",1743,
1,4,Aardenburg,Aalst Wilhelmus,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",1695,
2,2,Kalslagen ber,Aalst Cornelius van,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",1717,
3,3,Vuren en Dalem,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",1704,
3,3,Sommelsdijk,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",1706,


In [58]:
childs.to_csv(folderlink+folder_output+'minister_info.csv', sep=';', encoding='utf-8', index=False)

In [59]:
os.remove(output_txt)
os.remove(output_csv)